# FFIG - A foreign function inteface generator for `C++`

### For Project Managers
A language like Python can be written to resemble executable pseudo-code and is a great langauge for defining high-level acceptance criteria. 

Developer engagement is easy when requirements are code.

---

### For Clients 
It's easy to put together interactive demos with Python. Jupyter notebook allows one to display graphs, images, videos and tabulated data. 

The ability to change demos on the fly in response to client queries is seriously impressive.

---

### For Developers
I can break the edit-compile-link-test cycle with an interpreted scripting language. 

Once the design is right I can drive it into `C++`.

---

### Design

* Write some `C++` code.
* Define a `C-API` for binary compatibility.
* Solving problems with ownership.
* Define classes and functions that use a Foreign Function Interface to communicate with `C`.

---

### Code-generation

* Parse `C++` code.
* Transform parsed source code into a easy-to use form.
* Define a template that transforms parsed source code into `C`
* Define a template that transforms parsed source code into classes, functions and FFI calls.

---

### Extras

* Supporting Python 3.
* Supporting C++.

---